In [1]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

np.set_printoptions(precision=4, suppress=True)

In [2]:
def unit(v):
  return v / np.linalg.norm(v)

In [3]:
# original 3D data
X = np.array([
    [1, 2, 0.5],
    [2, 4, 1],
    [3, 6, 1.5],
    [4, 8, 2],
    [5, 10, 2.5],
]).T

X

array([[ 1. ,  2. ,  3. ,  4. ,  5. ],
       [ 2. ,  4. ,  6. ,  8. , 10. ],
       [ 0.5,  1. ,  1.5,  2. ,  2.5]])

In [4]:
# orthonormal basis (w1, w2, w3)
w1 = np.array([1, 1.2, 1])
w2 = np.cross(w1, [0, 0, 1])
w3 = np.cross(w1, w2)

w1 = unit(w1)
w2 = unit(w2)
w3 = unit(w3)

w1, w2, w3

(array([0.5392, 0.647 , 0.5392]),
 array([ 0.7682, -0.6402,  0.    ]),
 array([ 0.3452,  0.4142, -0.8422]))

In [5]:
c1 = X.T @ w1
c2 = X.T @ w2
c3 = X.T @ w3

In [6]:
proxies_w1 = w1.reshape(-1, 1) @ c1.reshape(1, -1)
proxies_w2 = w2.reshape(-1, 1) @ c2.reshape(1, -1)
proxies_w3 = w3.reshape(-1, 1) @ c3.reshape(1, -1)

In [7]:
X_reconstructed = proxies_w1 + proxies_w2 + proxies_w3

X
X_reconstructed
np.mean((X - X_reconstructed)**2)

array([[ 1. ,  2. ,  3. ,  4. ,  5. ],
       [ 2. ,  4. ,  6. ,  8. , 10. ],
       [ 0.5,  1. ,  1.5,  2. ,  2.5]])

array([[ 1. ,  2. ,  3. ,  4. ,  5. ],
       [ 2. ,  4. ,  6. ,  8. , 10. ],
       [ 0.5,  1. ,  1.5,  2. ,  2.5]])

np.float64(3.1636609219800996e-31)

In [8]:
fig = go.Figure()

# original
_ = fig.add_trace(go.Scatter3d(
    x=X[0], y=X[1], z=X[2],
    mode='markers',
    name='Original',
    marker=dict(size=5)
))

# reconstructed
_ = fig.add_trace(go.Scatter3d(
    x=X_reconstructed[0], y=X_reconstructed[1], z=X_reconstructed[2],
    mode='markers',
    name='Reconstructed',
    marker=dict(size=10, symbol='cross', color='orange')
))

# basis lines
for w, name in zip([w1, w2, w3], ['w1', 'w2', 'w3']):
  _ = fig.add_trace(go.Scatter3d(
      x=[0, w[0]*5], y=[0, w[1]*5], z=[0, w[2]*5],
      mode='lines+text',
      name=name
  ))

# coordinate axes
axes = [
    ([0, 6], [0, 0], [0, 0]),  # x-axis
    ([0, 0], [0, 6], [0, 0]),  # y-axis
    ([0, 0], [0, 0], [0, 6])   # z-axis
]
for a in axes:
  _ = fig.add_trace(go.Scatter3d(
      x=a[0], y=a[1], z=a[2],
      mode='lines',
      line=dict(color='black', width=4),
      showlegend=False
  ))

# XY plane
plane_x, plane_y = np.meshgrid(np.linspace(-2, 6, 2), np.linspace(-2, 6, 2))
plane_z = np.zeros_like(plane_x)
_ = fig.add_trace(go.Surface(
    x=plane_x, y=plane_y, z=plane_z,
    colorscale=[[0, 'rgba(200,200,200,0.3)'], [1, 'rgba(200,200,200,0.3)']],
    showscale=False,
    name='XY plane',
    hoverinfo='skip'
))

fig.update_layout(
    width=800,
    height=800,
    scene_camera=dict(
        eye=dict(x=0, y=0, z=2),
        up=dict(x=0, y=1, z=0),
        center=dict(x=0, y=0, z=0)
    ),
    scene=dict(aspectmode='cube'),
    title="3D Projection and Reconstruction"
)

# variance explained

In [9]:
v1 = np.var(c1)
v2 = np.var(c2)
v3 = np.var(c3)
total = v1 + v2 + v3

v1/total, v2/total, v3/total
np.cumsum([v1/total, v2/total, v3/total])

(np.float64(0.8421926910299004),
 np.float64(0.04996096799375489),
 np.float64(0.10784634097634474))

array([0.8422, 0.8922, 1.    ])